In [22]:
import os

In [23]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [12]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware

from langserve import add_routes

from base.llm_model import get_hf_llm
from rag.main import build_rag_chain, InputQA, OutputQA

In [13]:
%pwd

'C:\\Users\\Lenovo\\learning\\RAGwithLangchain\\src'

# SELF

In [16]:
llm = get_hf_llm(temperature=0.9)
genai_docs = "../data_source/generative_ai"

Loading checkpoint shards: 100%|██████████| 2/2 [00:19<00:00,  9.68s/it]
Device set to use cpu


In [19]:
genai_chain = build_rag_chain(llm, data_dir=genai_docs, data_type='pdf')

Loading PDFs: 100%|██████████| 6/6 [02:16<00:00, 22.78s/file]
C:\Users\Lenovo\anaconda3\envs\basellmchatbot\lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [27]:
input_data = InputQA(question="What is the capital of France?")
output_data = genai_chain.invoke(input_data.question)

Exception ignored in: <function WeakSet.__init__.<locals>._remove at 0x000002903EDFC820>
Traceback (most recent call last):
  File "C:\Users\Lenovo\anaconda3\envs\basellmchatbot\lib\_weakrefset.py", line 40, in _remove
    self = selfref()
KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
output_data.answer

# Application

In [ ]:
# App - FastAPI
app = FastAPI(
    title="LangChain Server",
    version="1.0",
    description="A simple api server using LangChain's Runnable interfaces"
)

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
    expose_headers=["*"]
)

In [ ]:
# Routes - FastAPI
@app.get("/check")
async def check():
    return {"status": "ok"}

@app.post("/generative_ai", response_model=OutputQA)
async def generative_ai(inputs: InputQA):
    answer = genai_chain.invoke(inputs.question)
    return {"answer": answer}

In [ ]:
# Langserve Routes - Playground
add_routes(app,
    genai_chain,
    playground_type="default",
    path="/generative_ai")